In [2]:
import csv
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


def DisplayOriginalAndAugumentedImages(original, flipped, shifted):
    plt.figure(figsize=(9, 5))
    plt.subplot(1, 3, 1)
    fig = plt.imshow(original)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.title("Original")
    plt.subplot(1, 3, 2)
    fig = plt.imshow(flipped.squeeze())
    plt.title("Flipped")
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.subplot(1, 3, 3)
    fig = plt.imshow(shifted.squeeze())
    plt.title("Shifted")
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)

def ShiftImageRandomly(image, steering_angle, x_shift_range, y_shift_range):
    #get random values for horizontal and vertical shifts (in pixels)
    x_shift = x_shift_range * (np.random.uniform() - 0.5)
    y_shift = y_shift_range * (np.random.uniform() - 0.5)
    # calculate new steering angle.  0.4 is the steering angle adjustment for maximum horizontal shift to one side (x_shift_range)
    new_steering_angle = steering_angle + x_shift/x_shift_range * 0.4
    # use Affine transformation for image shifting
    Trans_M = np.float32([[1, 0, x_shift],[0, 1, y_shift]])
    shifted_image = cv2.warpAffine(image,Trans_M,(320,160))
    return shifted_image, new_steering_angle

def AddImageAndSteeringAngle(images, steering_angles, current_training_data_dir, recorded_image_file_path, steering_angle):
    current_image_file_path = os.path.join(current_training_data_dir, 'IMG', recorded_image_file_path.split('/')[-1])
    if(not os.path.isfile(current_image_file_path)):
        print ("File does not exist", current_image_file_path)
    else:    
        originalImage = cv2.imread(current_image_file_path)
        image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
        
        images.append(image)
        steering_angles.append(steering_angle)
        
        ### randomly shift original image
        shifted_image, shifted_angle = ShiftImageRandomly(image, steering_angle, x_shift_range=50, y_shift_range=40)
        images.append(shifted_image)
        steering_angles.append(shifted_angle)
        
        ### add flipped image and invertted angle to emulate reverse direction case
        flipped_image = cv2.flip(image, 1)
        images.append(flipped_image)
        steering_angles.append(steering_angle * -1.0)
        
        DisplayOriginalAndAugumentedImages(image, shifted_image, flipped_image)


def DisplaySteeringAnglesHistohram(title, angles):
    plt.hist(angles, bins=80, rwidth=0.7, histtype='bar')
    plt.xlim(xmin=-0.7, xmax=0.7)
    plt.title(title)
    plt.xlabel("Steering angle")
    plt.ylabel("Number of samples")
    plt.show()  
    

def DisplayImagesInTable(img_set, label_set) :
    max_columns=3
    rows = (len(label_set) / max_columns) + 1
    plt.figure(figsize=(9, 5))
    for i in range(3):
        label = label_set[i]
        # get correspondent image from image set    
        image = img_set[i].squeeze()
        # Each subplot is an example of traffic sign class with its name    
        plt.subplot(rows, max_columns, i + 1)
        plt.title(label)
        fig = plt.imshow(image)
        # hide image size scale axes
        #fig.axes.get_xaxis().set_visible(False)
        #fig.axes.get_yaxis().set_visible(False)    

def DisplayAllImagesOfWaypoint(waypoint):
    waypoint_images = [] 
    camera_angles = [] 
    waypoint = data[0]
    data_dir = os.path.join(training_data_root_path, waypoint[0])
    angle = float(waypoint[1][3])

    AddImageAndSteeringAngle(waypoint_images, camera_angles, data_dir, waypoint[1][0], angle)
    AddImageAndSteeringAngle(waypoint_images, camera_angles, data_dir, waypoint[1][1], angle + stering_angle_correction)
    AddImageAndSteeringAngle(waypoint_images, camera_angles, data_dir, waypoint[1][2], angle - stering_angle_correction)
    
    #DisplayImagesInTable(waypoint_images, camera_angles)                        
        
    
training_data_root_path = './data/'
training_data_dirs = [ directory for directory in os.listdir(training_data_root_path) if os.path.isdir(os.path.join(training_data_root_path, directory)) ]
print ('Parsing data samples from following directories:', training_data_dirs)

stering_angle_correction = 0.25 
data_sample_augumentation_factor = 9 ### 1 data sampple = 3 cameras x (1 + augumentation_types_number)

data = []
steer_angles=[]
for data_dir in training_data_dirs:
    with open(os.path.join(training_data_root_path, data_dir, 'driving_log.csv')) as csvfile:
        reader = csv.reader(csvfile)
        for table_row in reader:
            steering_angle = float(table_row[3])
            # tripple coin flip to decide whether to 'zero steering angle' way point
            include_zero_angle_image = np.random.randint(0,2) and np.random.randint(0,2) and np.random.randint(0,2)
            if(steering_angle != 0.0 or include_zero_angle_image):
                data.append((data_dir, table_row))
                steer_angles.append(steering_angle)
                
print("Number of recorded way points: ", len(data))
training_data, validation_data = train_test_split(data, test_size=0.2)            
            
print("Number of imgaes in training dataset: ", len(training_data) * data_sample_augumentation_factor)
print("Number of imgaes in validation dataset: ", len(validation_data) * data_sample_augumentation_factor)
                

#DisplaySteeringAnglesHistohram(steer_angles)

DisplayAllImagesOfWaypoint(data[0])


Parsing data samples from following directories: ['2_laps_center_reverse', 'corners', 'provided', 'recoveries']
Number of recorded way points:  21786
Number of imgaes in training dataset:  156852
Number of imgaes in validation dataset:  39222
